In [1]:
import requests
import os
import json
import pandas as pd
from tqdm import tqdm

In [11]:
base_folder = 'dublin_dl_birds'

In [36]:
def xeno_download(search_string, abbrev_string = None, top_n = 250, ssp = None, not_ssp = None):
    folder_sv = base_folder+'/'+search_string
    find_bracket = (search_string.find(' ('))
    if (ssp is not None) & (find_bracket == -1):
        folder_sv = folder_sv + ' ('+ssp.capitalize()+')'
    folder_sv = folder_sv +'/'
    if not os.path.isdir(folder_sv):
        os.mkdir(folder_sv)   
    if abbrev_string is None:
        if ssp is not None:
            abbrev_string = (search_string+ssp).replace(' ', '_').lower()
        else:
            abbrev_string = search_string.replace(' ', '_').lower()
    if find_bracket != -1:
        search_string = search_string[:find_bracket]
    search_str = search_string.replace(' ', '%20').lower()
    if ssp is not None:
        url = 'https://www.xeno-canto.org/api/2/recordings?query=ssp:'+ssp
    else:
        url = 'https://www.xeno-canto.org/api/2/recordings?query='+search_str
    doc = requests.get(url)
    with open(abbrev_string+'-query.json', 'wb') as f:
        f.write(doc.content)
    
    # Get the json entries from your downloaded json
    jsonFile = open(abbrev_string+'-query.json', 'r')
    values = json.load(jsonFile)
    jsonFile.close()

    # Create a pandas dataframe of records & convert to .csv file
    record_df = pd.DataFrame(values['recordings'])
    #record_df.to_csv('xc-'+abbrev_string+'.csv', index=False)
    record_df = record_df[record_df.en.str.startswith(search_string)]
    if not_ssp is not None:
        record_df = record_df[record_df.ssp != not_ssp]
    #return record_df
    record_df_uk_ire = record_df[[rc in ['Ireland', 'United Kingdom'] for rc in record_df.cnt]]
    if len(record_df_uk_ire) >= top_n:
        record_df = record_df_uk_ire.iloc[:top_n]
    else:
        record_df_else = record_df[[rc not in ['Ireland', 'United Kingdom'] for rc in record_df.cnt]]
        record_df = record_df_uk_ire.append(record_df_else.iloc[:(top_n-len(record_df_uk_ire))])
    #return record_df
    # Make wget input file
    url_list = []
    for file in record_df['file'].tolist():
        url_list.append('https:{}'.format(file))
    with open('xc-'+abbrev_string+'-urls.txt', 'w+') as f:
        for item in url_list:
            f.write("{}\n".format(item))
            
    txt_file = open('xc-'+abbrev_string+'-urls.txt', 'r')
    url_list = [s.replace('\n', '') for s in txt_file.readlines()]
    
    url_list = url_list[:top_n]
     
    for url_i in url_list:
        filename_sv = folder_sv+'xc'+url_i.split('/')[3]+'.mp3'
        if filename_sv in os.listdir(folder_sv):
            continue
        doc = requests.get(url_i)
        with open(filename_sv, 'wb') as f:
            f.write(doc.content)

In [13]:
species_df = pd.read_excel('species list ireland - dun laoghaire.xlsx')

In [14]:
species_df.loc[species_df.ssp.isna(),'ssp'] = None
species_df = species_df[species_df.use_train == 1]
species_df

,species,num_recordings,folder_label,ssp,use_train
0,Mediterranean Gull,NaN,Mediterranean Gull,NaN,1
1,Gannet,NaN,Northern Gannet,NaN,1
6,Grey Heron,NaN,Grey Heron,NaN,1
7,Starling,NaN,Common Starling,NaN,1
8,Herring Gull,NaN,European Herring Gull,NaN,1
12,Black Bird,NaN,Common Blackbird,NaN,1
13,Robin,NaN,European Robin,NaN,1
14,Great Tit,NaN,Great Tit,NaN,1
15,Blue Tit,NaN,Eurasian Blue Tit,NaN,1
16,Coal Tit,NaN,Coal Tit,NaN,1


In [51]:
species_df = species_df[1:]
species_df

,species,num_recordings,folder_label,ssp,use_train
17,Chaffinch,NaN,Common Chaffinch,NaN,1
18,Magpie,NaN,Eurasian Magpie,NaN,1
19,Goldfinch,NaN,European Goldfinch,NaN,1
20,House Sparrow,NaN,House Sparrow,NaN,1
21,Greenfinch,NaN,European Greenfinch,NaN,1


In [29]:
#df_tmp = xeno_download('Mediterranean Gull', top_n = 500)

In [32]:
#df_tmp

In [52]:
for indx, row in tqdm(species_df.iterrows()):
    #print(row.folder_label)
    xeno_download(row.folder_label, top_n = 100)

5it [04:21, 52.31s/it]


In [48]:
import ffmpeg
from pydub import AudioSegment

In [49]:
wav_check = base_folder
labels = os.listdir(wav_check)
#print(labels)
for lab in labels:
    for sound_f in os.listdir(os.path.join(wav_check,lab)):
        pre, ext = os.path.splitext(os.path.join(wav_check,lab,sound_f))
        #print(ext)
        if ext == '.wav':
            continue
        s_temp   = AudioSegment.from_file(pre+ext)
        mono_audios = s_temp.split_to_mono()
        if len(mono_audios) >=2:
            mono_audios[0].export(pre+'_left.wav', format="wav")
            mono_audios[1].export(pre+'_right.wav', format="wav")
        else:
            mono_audios[0].export(pre+'.wav', format="wav")

In [50]:
import re
labels = os.listdir(wav_check)
for lab in labels:
    dirname = wav_check+'/'+lab
    for sound_f in os.listdir(dirname):
        if re.search('.wav', sound_f) == None:
            os.remove(os.path.join(dirname, sound_f))

In [51]:
birds = os.listdir(base_folder)
count = 0
count_wav = 0
count_list = []
print('Bird audio files')
for bird in birds:
    bird_list = os.listdir(os.path.join(base_folder, bird))
    bird_count = len(bird_list)
    print('\t', bird, ':\t\t', bird_count, sep = '')
    bird_count_wav = len([s for s in bird_list if s.endswith('wav')])
    print('  of which .wav files:', bird_count_wav)
    count += bird_count
    count_wav += bird_count_wav
    count_list.append(bird_count)
print('Total: ', count)
print('Total .wav: ', count_wav)

Bird audio files
	Common Buzzard:		166
  of which .wav files: 166
	Common Kestrel:		165
  of which .wav files: 165
	Common Snipe:		179
  of which .wav files: 179
	Eurasian Curlew:		146
  of which .wav files: 146
	European Herring Gull:		174
  of which .wav files: 174
	European Robin:		184
  of which .wav files: 184
	Meadow Pipit:		170
  of which .wav files: 170
	Mute Swan:		177
  of which .wav files: 177
	Northern Lapwing:		182
  of which .wav files: 182
	Rook:		175
  of which .wav files: 175
	Tundra Swan:		133
  of which .wav files: 133
	Tundra Swan (Bewicks):		130
  of which .wav files: 130
Total:  1981
Total .wav:  1981
